<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-1">Dependencies</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2">Functions</a></span></li><li><span><a href="#Paths" data-toc-modified-id="Paths-3">Paths</a></span></li><li><span><a href="#Main" data-toc-modified-id="Main-4">Main</a></span><ul class="toc-item"><li><span><a href="#CBTTC" data-toc-modified-id="CBTTC-4.1">CBTTC</a></span></li><li><span><a href="#Clean-up-mupexi-files-to-include-mutations-that-are-overlapped-from-strelka-and-mutect2-for-cbttc" data-toc-modified-id="Clean-up-mupexi-files-to-include-mutations-that-are-overlapped-from-strelka-and-mutect2-for-cbttc-4.2">Clean up mupexi files to include mutations that are overlapped from strelka and mutect2 for cbttc</a></span></li><li><span><a href="#TARGET" data-toc-modified-id="TARGET-4.3">TARGET</a></span></li></ul></li></ul></div>

# Dependencies

In [1]:
library(maftools)

# Functions

# Paths

In [2]:
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/IPD2/Manifests/"
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/IPD2/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/IPD2/Plots/"

# Main

In [3]:
load(file = paste0(datapath,"snv/total_mut_manifest_estimate.RData"))

## CBTTC

In [4]:
CBTTC_mafsummary <- read.table(file = paste0(datapath, "snv/CBTTC_mafsummary_overlapped_3VAF.txt"),
                        sep = "\t", check.names = F,
                        header = TRUE, stringsAsFactors = FALSE)

In [5]:
mymaf_CBTTC <- read.maf(maf =  paste0(datapath, "snv/CBTTC_maf_overlapped_3VAF.maf"),
                      useAll = TRUE, verbose = FALSE,
                      vc_nonSyn = c("Frame_Shift_Del", "Frame_Shift_Ins", 
                                    "Nonsense_Mutation","Nonstop_Mutation", 
                                    "In_Frame_Del","In_Frame_Ins", 
                                    "Missense_Mutation"))

In [6]:
CBTTC_data_maf <- mymaf_CBTTC@data

In [7]:
CBTTC_data_maf$Tumor_Sample_Barcode <- as.character(CBTTC_data_maf$Tumor_Sample_Barcode)

Add sample id to maf file

In [8]:
CBTTC_data_maf_sampleid <- merge(CBTTC_mafsummary[,c("sample_id", "Tumor_Sample_Barcode")], 
                                 CBTTC_data_maf, by = "Tumor_Sample_Barcode")

In [9]:
length(unique(CBTTC_data_maf_sampleid$Tumor_Sample_Barcode))

[1] 796

subset to samples included in total_mut_manifest_estimate_quanti

In [10]:
ls()

[1] "CBTTC_data_maf"              "CBTTC_data_maf_sampleid"    
[3] "CBTTC_mafsummary"            "datapath"                   
[5] "manifestpath"                "mymaf_CBTTC"                
[7] "plotpath"                    "total_mut_manifest_estimate"

In [11]:
CBTTC_data_maf <- CBTTC_data_maf_sampleid[CBTTC_data_maf_sampleid$sample_id %in% total_mut_manifest_estimate$sample_id,]

In [12]:
head(CBTTC_data_maf$sample_id)

[1] "7316-2558" "7316-3299" "7316-3299" "7316-3299" "7316-3299" "7316-3299"

In [13]:
length(unique(CBTTC_data_maf$Tumor_Sample_Barcode))

[1] 470

## Clean up mupexi files to include mutations that are overlapped from strelka and mutect2 for cbttc

CBTTC manifest that optitype and mutect2 vcf files are matched with sample ids and all. Use this to map mupexi filenames to sample_id

In [14]:
opti_mutect2_cbttc <- read.table(file = paste0(manifestpath, "mutect2_optitype_manifest_cbttc.txt"),
                        sep = "\t", check.names = F, 
                        header = TRUE, stringsAsFactors = FALSE)

In [15]:
dim(opti_mutect2_cbttc)

[1] 512  66

In [16]:
head(opti_mutect2_cbttc$name)

[1] "b5a7b96d-940c-475a-a439-32b898026903.recode.vcf"
[2] "9ce2828c-bbcf-4112-a4cb-95cc69c72c81.recode.vcf"
[3] "aa08d3dd-3ba7-4852-8929-eaa68c1e5856.recode.vcf"
[4] "bd321ccc-ce04-4164-91fb-676105eef65e.recode.vcf"
[5] "e31d632e-7cff-41de-b4da-57f5bf53a5c8.recode.vcf"
[6] "d9eaced6-2f46-4876-90cd-3c2ff990615e.recode.vcf"

In [17]:
datapath

[1] "/Users/anabbi/OneDrive - UHN/Documents/IPD2/Data/"

In [18]:
mountpath <- "/Users/anabbi/Desktop/H4H/immpedcan/antigen_prediction/output/"

In [22]:
flelist <- list.files(mountpath, pattern = "mupexi")

In [23]:
length(flelist)

[1] 603

In [24]:
length(unique(CBTTC_data_maf$sample_id))

[1] 470

In [25]:
for(f in flelist){
    print(f)
    #get the filename
    myfilename <- gsub(".mupexi", "", f)
    #get sample_id by grepping filename from manifest file
    mysample_id <- opti_mutect2_cbttc$sample_id[grepl(myfilename, opti_mutect2_cbttc$name)]
    #skip if sample_id is not in CBTTC_data_maf
    if(sum(CBTTC_data_maf$sample_id == mysample_id) == 0){ 
        message("sample_id not found")
        next}
    # get overlapped vars for the sample_id
    myoverlappedmutations <- CBTTC_data_maf[CBTTC_data_maf$sample_id == mysample_id,]
    
    #read the mupexi file
    mymupexi <- read.table(file = paste0(mountpath, f),
                        sep = "\t", check.names = F, comment.char = "#",
                        header = TRUE, stringsAsFactors = FALSE)
    #skip if genes in mupexi output arent in overlapped mutations
    if(sum(mymupexi$Gene_ID %in% myoverlappedmutations$Gene) == 0){ 
        message("Genes in mupexi not found in overlapped variants")
        next}
    #subset to overlapped vars only using Ensembl IDs
    mupexi_genes <- mymupexi[ mymupexi$Gene_ID %in% myoverlappedmutations$Gene,]
    mupexi_genes_AA <- mupexi_genes[ mupexi_genes$Amino_Acid_Change %in% myoverlappedmutations$Amino_acids,]
    # annotate binders for Norm and Mut
    mupexi_genes_AA$Norm_MHCbinder <- NA
    mupexi_genes_AA$Norm_MHCbinder[mupexi_genes_AA$Norm_MHCrank_BA <= 0.5] <- "SB"
    mupexi_genes_AA$Norm_MHCbinder[mupexi_genes_AA$Norm_MHCrank_BA > 0.5 &
                                   mupexi_genes_AA$Norm_MHCrank_BA <= 2] <- "WB"


    mupexi_genes_AA$Mut_MHCbinder <- NA
    mupexi_genes_AA$Mut_MHCbinder[mupexi_genes_AA$Mut_MHCrank_BA <= 0.5] <- "SB"
    mupexi_genes_AA$Mut_MHCbinder[mupexi_genes_AA$Mut_MHCrank_BA > 0.5 &
                                  mupexi_genes_AA$Mut_MHCrank_BA <= 2] <- "WB"
    
      
    #Include all peptides
    mupexi_genes_AA_binders <- mupexi_genes_AA #[ !is.na(mupexi_genes_AA$Mut_MHCbinder),]
    
    #skip if no SB or WB in mupexi output arent in overlapped mutations
    if(nrow(mupexi_genes_AA_binders) == 0){ 
        message("No SB or WB found in Mut_MHC")
        next} 
    # add sample_id as variable
    mupexi_genes_AA_binders$sample_id <- mysample_id
    write.table(mupexi_genes_AA_binders,
           file = paste0(datapath, "Mupexi/", mysample_id, "_overlappedvar_all.mupexi"), sep = "\t", quote = F, row.names = F)
    
}

[1] "00267954-7696-4d50-b48b-75d0ae76dafc.mupexi"


sample_id not found



[1] "00382118-5ea5-447b-9ec3-b36ad8d2a2f7.mupexi"


sample_id not found



[1] "0062aa93-2238-46cc-82fb-a4a1fe8eb19e.mupexi"
[1] "0167ee47-7441-4965-8cd8-c7b607fcca22.mupexi"
[1] "01a07f01-2f87-4b88-9aeb-d2bffdb75a24.mupexi"
[1] "01c6d153-7b01-418a-be36-4f0dbee5e85c.mupexi"
[1] "041816ac-abd8-4f4a-af58-4b186bd5dc03.mupexi"
[1] "0436934f-3c8f-490e-bba8-f04ac7dcb58c.mupexi"
[1] "0449d846-0d30-4749-95e0-51085f07a4ed.mupexi"
[1] "04b96cbf-d1c2-434a-ab0e-0b2931414964.mupexi"
[1] "053dac4d-c91e-457b-8194-6272e0affeba.mupexi"
[1] "05c2f817-353b-45af-b386-8214339d498f.mupexi"
[1] "074c52e2-5774-4695-8e0c-78e3b3856457.mupexi"


...

[1] "fcb6fbf5-6045-4076-97c5-67f5027a44ca.mupexi"


sample_id not found



[1] "fd982d34-29d2-450b-a9f5-541b8c7811a6.mupexi"
[1] "fdf01de6-dba8-4ef8-b7b5-76ee8e53a00d.mupexi"
[1] "ff436ec0-492c-414a-8564-74adaf4c9fac.mupexi"


sample_id not found



[1] "ffdd6cc0-0b9f-488b-b5d0-6ed5908e98ea.mupexi"


## TARGET

In [23]:
mymaf_gdc <- read.maf(maf =  paste0(datapath, "snv/GDC_allmaf_3VAF_fl_dedup_sampleid_KB.maf"),
                      useAll = TRUE, verbose = FALSE,
                      vc_nonSyn = c("Frame_Shift_Del", "Frame_Shift_Ins", 
                                    "Nonsense_Mutation","Nonstop_Mutation", 
                                    "In_Frame_Del","In_Frame_Ins", 
                                    "Missense_Mutation"))

In [24]:
mymaf_gdc_data<- mymaf_gdc@data

In [25]:
mymaf_gdc_data$Tumor_Sample_Barcode <- as.character(mymaf_gdc_data$Tumor_Sample_Barcode)

Subset to TARGET only

In [26]:
mymaf_target_data <- mymaf_gdc_data[grepl("TARGET", mymaf_gdc_data$Tumor_Sample_Barcode),]

subset to samples included in total_mut_manifest_estimate_quanti

In [27]:
mymaf_target_data <- mymaf_target_data[mymaf_target_data$Tumor_Sample_Barcode %in% 
                                       total_mut_manifest_estimate$sample_id,]

In [28]:
colnames(mymaf_target_data)[colnames(mymaf_target_data) == "Tumor_Sample_Barcode"] <- "sample_id"

In [29]:
length(unique(mymaf_target_data$sample_id))

[1] 98

read in optitype-mutect2 manifest

In [30]:
opti_mutect2_target <- read.table(file = paste0(manifestpath, "mutect2_optitype_manifest_target.txt"),
                        sep = "\t", check.names = F, 
                        header = TRUE, stringsAsFactors = FALSE)

In [31]:
for(f in flelist){
    print(f)
    #get the filename
    myfilename <- gsub(".mupexi", "", f)
    #get sample_id by grepping filename from manifest file
    mysample_id <- opti_mutect2_target$sample_id.x[grepl(myfilename, opti_mutect2_target$name.y)]
    #skip if sample_id is not in CBTTC_data_maf
    if(sum(mymaf_target_data$sample_id == mysample_id) == 0){ 
        message("sample_id not found")
        next}
    # get overlapped vars for the sample_id
    myoverlappedmutations <- mymaf_target_data[mymaf_target_data$sample_id == mysample_id,]
    
    #read the mupexi file
    mymupexi <- read.table(file = paste0(mountpath, f),
                        sep = "\t", check.names = F, comment.char = "#",
                        header = TRUE, stringsAsFactors = FALSE)
    #skip if genes in mupexi output arent in overlapped mutations
    if(sum(mymupexi$Gene_ID %in% myoverlappedmutations$Gene) == 0){ 
        message("Genes in mupexi not found in overlapped variants")
        next}
    #subset to overlapped vars only using Ensembl IDs
    mupexi_genes <- mymupexi[ mymupexi$Gene_ID %in% myoverlappedmutations$Gene,]
    mupexi_genes_AA <- mupexi_genes[ mupexi_genes$Amino_Acid_Change %in% myoverlappedmutations$Amino_acids,]
    # annotate binders for Norm and Mut
    mupexi_genes_AA$Norm_MHCbinder <- NA
    mupexi_genes_AA$Norm_MHCbinder[mupexi_genes_AA$Norm_MHCrank_BA <= 0.5] <- "SB"
    mupexi_genes_AA$Norm_MHCbinder[mupexi_genes_AA$Norm_MHCrank_BA > 0.5 &
                                   mupexi_genes_AA$Norm_MHCrank_BA <= 2] <- "WB"


    mupexi_genes_AA$Mut_MHCbinder <- NA
    mupexi_genes_AA$Mut_MHCbinder[mupexi_genes_AA$Mut_MHCrank_BA <= 0.5] <- "SB"
    mupexi_genes_AA$Mut_MHCbinder[mupexi_genes_AA$Mut_MHCrank_BA > 0.5 &
                                  mupexi_genes_AA$Mut_MHCrank_BA <= 2] <- "WB"
    
    #Include only those that Mut_MHCbinder is eather WB or SB
    mupexi_genes_AA_binders <- mupexi_genes_AA#[ !is.na(mupexi_genes_AA$Mut_MHCbinder),]
    #skip if no SB or WB in mupexi output arent in overlapped mutations
    if(nrow(mupexi_genes_AA_binders) == 0){ 
        message("No SB or WB found in Mut_MHC")
        next} 
    # add sample_id as variable
    mupexi_genes_AA_binders$sample_id <- mysample_id    
    write.table(mupexi_genes_AA_binders,
           file = paste0(datapath, "Mupexi/", mysample_id, "_overlappedvar_all.mupexi"), sep = "\t", quote = F, row.names = F)
    
}